## COMP5625M Deep Learning - Practical Assessment [100 Marks]


<div class="logos"><img src="Comp5625M_logo.jpg" width="220px" align="right"></div>

This assessment is divided into two parts:
> 1. Image classification using a convolutional neural network (CNN), training your model, evaluating your model, understanding fine-tuning, handling overfitting  [60 Marks]
> 2. Use of a Recurrent Neural Network (RNN) to predict texts for image captioning, and use of large language models (LLMs) for this task [40 Marks]


The maximum number of marks for each part is shown in the section headers. As indicated in the main heading above, the overall assessment carries a maximum of 100 marks.

This summative assessment is weighted 50% of the final grade for the module.


### Motivation 

Through this coursework, you will:

> 1. Understand and implement a convolutional neural network (CNN) and see how this can be used for a classification problem. 
> 2. Practice building, evaluating, and finetuning your CNN on an image dataset from development to testing stage. 
> 3. You will learn to tackle overfitting problem using strategies such as data augmentation and drop out.
> 4. Compare your model performance and accuracy with others, such as the leaderboard on Kaggle.
> 5. Use a Recurrent Neural Network (RNN) to predict the caption of an image from established word vocabularies.
> 6. Understand and visualise text predictions for a given image.
> 7. Learn to use transformers for image embedding and transformer-based large language models for text embedding learning.


### Setup and resources 

You must work using this provided template notebook.

Having a GPU will speed up the training process. See the provided document on Minerva about setting up a working environment for various ways to access a GPU.

Please implement the coursework using **Python and PyTorch**, and refer to the notebooks and exercises provided.


### Submission

Please submit the following:

> 1. Your completed Jupyter notebook file, without removing anything in the template, in **.ipynb format.**
> 2. The **.html version** of your notebook; File > Download as > HTML (.html). Check that all cells have been run and all outputs (including all graphs you would like to be marked) displayed in the .html for marking.
> 3. Your selected image from section 2.4.2 "Failure analysis"

Final note:

> **Please display everything that you would like to be marked. Under each section, put the relevant code containing your solution. You may re-use functions you defined previously, but any new code must be in the relevant section.** Feel free to add as many code cells as you need under each section.


Your student username (for example, ```sc15jb```): 

--> double click to respond

Your full name: 

--> double click to respond

### Part I: Image Classification [60 marks]

#### Dataset
This coursework will use a subset of images from Tiny ImageNet, which is a subset of the [ImageNet dataset](https://www.image-net.org/update-mar-11-2021.php). Our subset of Tiny ImageNet contains **30 different categories**, we will refer to it as TinyImageNet30. The training set has 450 resized images (64x64 pixels) for each category (13,500 images in total). You can download the training and test set from the Kaggle website:

>[Direct access of data is possible by clicking here, please use your university email to access this](https://leeds365-my.sharepoint.com/:u:/g/personal/scssali_leeds_ac_uk/ESF87mN6kelIkjdISkaRow8BublW27jB-P8eWV6Rr4rxtw?e=SPASDB)

>[To submit your results on the Kaggle competition. You can also access data here](https://www.kaggle.com/t/d77d518158104932a434a8954ec9ebc2)

To access the dataset, you will need an account on the Kaggle website. Even if you have an existing Kaggle account, please carefully adhere to these instructions, or we may not be able to locate your entries:

> 1. Use your **university email** to register a new account.
> 2. Set your **Kaggle account NAME** to your university username, for example, ``sc15jb`` (see the ``note`` below)

``Note:`` If the name is already taken in Kaggle then please use a similar pseudo name and add a note in your submission with the name you have used in Kaggle. 

#### Submitting your test result to Kaggle leaderboard 
The class Kaggle competition also includes a blind test set, which will be used in Question 1 for evaluating your custom model's performance on a test set. The competition website will compute the test set accuracy, as well as position your model on the class leaderboard. More information is provided in the related section below.


## Required packages

[1] [numpy](http://www.numpy.org) is package for scientific computing with python

[2] [h5py](http://www.h5py.org) is package to interact with compactly stored dataset

[3] [matplotlib](http://matplotlib.org) can be used for plotting graphs in python

[4] [pytorch](https://pytorch.org/docs/stable/index.html) is library widely used for bulding deep-learning frameworks

Feel free to add to this section as needed. Examples for importing some libraries is provided for you below.

You may need to install these packages using [pip](https://pypi.org/project/opencv-python/) or [conda](https://anaconda.org/conda-forge/opencv).

In [ ]:
import cv2
import math

import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.hub import load_state_dict_from_url

from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# always check your version
print(torch.__version__)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

One challenge of building a deep learning model is to choose an architecture that can learn the features in the dataset without being unnecessarily complex. The first part of the coursework involves building a CNN and training it on TinyImageNet30. 

### **Overview of image classification:**

**1. Function implementation** [10 marks]

*   **1.1** PyTorch ```Dataset``` and ```DataLoader``` classes (4 marks)
*   **1.2** PyTorch ```Model``` class for 5-layer (up to 7 layers at most) CNN model (6 marks)

**2. Model training** [24 marks]
*   **2.1** Training on TinyImageNet30 dataset (6 marks)
*   **2.2** Generating confusion matrices and ROC curves (6 marks)
*   **2.3** Strategies for tackling overfitting (12 marks)
    *   **2.3.1** Data augmentation
    *   **2.3.2** Dropout
    *   **2.3.3** Hyperparameter tuning (e.g. changing learning rate)
            
**3. Model testing** [10 marks]
*   **3.1**   Testing your final model in (2) on test set - code to do this (4 marks)
*   **3.2**   Uploading your result to Kaggle  (6 marks)

**4. Model Fine-tuning on CIFAR10 dataset** [16 marks]
*   **4.1** Fine-tuning your model (initialise your model with pretrained weights from (2)) (6 marks)
*   **4.2** Fine-tuning model with frozen base convolution layers (6 marks)
*   **4.3** Compare complete model retraining with pretrained weights and with frozen layers. Comment on what you observe. (4 marks) 


<!-- **5. Model comparison** [16 marks]
*   **5.1**   Load pretrained AlexNet and finetune on TinyImageNet30 until model convergence (8 marks)
*   **5.2**   Compare the results of your model with pretrained AlexNet on the same validation set. Provide performance values (loss graph, confusion matrix, top-1 accuracy, execution time) (8 marks) -->
<!-- 
**6. Interpretation of results** (14 marks)
*   **6.1** Implement grad-CAM for your model and AlexNet (6 marks)
*   **6.2** Visualise and compare your results from your model and AlexNet (4 marks)
*   **6.3** Provide comment on (4 marks)
    - why the network predictions were correct or not correct in your predictions? 
    - what can you do to improve your results further?

**7. Residual connection for deeper network** (9 marks)
*   **7.1** Implement a few residual layers in AlexNet and retrain on TinyImageNet30. You can change network size if you wish. (6 marks)
*   **7.2** Comment on why such connections are important and why this impacted your results in terms of loss and accuracy (if it did!) (3 marks)

**Quality of your report** (2 marks) -->


## 1 Implement a CNN network and dataset class [10 marks]


### 1.1 Dataset class (4 marks)

Write a PyTorch ```Dataset``` class (an example [here](https://www.askpython.com/python-modules/pytorch-custom-datasets) for reference) which loads the TinyImage30 dataset and ```DataLoaders``` for training and validation sets.

In [ ]:
# TO COMPLETE

### 1.2 Define a CNN model class (6 marks)

<u>Create a new model class using a combination of:</u>
- Convolution layers
- Activation functions (e.g. ReLU)
- Maxpooling layers
- Fully connected layers 
- Loss function
- Optimiser

*Please note that the network should be at least a few layers for the model to perform well.*

In [1]:
# TO COMPLETE
# define a CNN Model class - ideally 5-7 convolutional layers

## 2 Model training [24 marks]


### 2.1 Train a CNN model - show loss and accuracy graphs side by side (6 marks)

Train your model on the TinyImageNet30 dataset. Split the data into train and validation sets to determine when to stop training. Use seed at 0 for reproducibility and test_ratio=0.2 (validation data)

Display the graph of training and validation loss over epochs and accuracy over epochs to show how you determined the optimal number of training epochs. A top-*k* accuracy implementation is provided for you below.

> Please leave the graph clearly displayed. Please use the same graph to plot graphs for both train and validation.


In [ ]:
# (HelperDL function) -- Define top-*k* accuracy (**new**)
def topk_accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)
    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [ ]:
#TO COMPLETE --> Running your CNN model class

In [ ]:
# Your graph

> Comment on your model and the results you have obtained. This should include the number of parameters for your models and briefly explain why one should use CNN. You could write a few advantages of using CNN over MLP.

### 2.2 Generating confusion matrix and ROC curves (6 marks)
- Use your CNN architecture with best accuracy to generate two confusion matrices, one for the training set and another for the validation set. Remember to use the whole validation and training sets, and to include all your relevant code. Display the confusion matrices in a meaningful way which clearly indicates what percentage of the data is represented in each position.
- Display an ROC curve for the two top and two bottom classes with area under the curve

In [ ]:
# Your code here!

### 2.3 Strategies for tackling overfitting (12 marks)
Using your (final) CNN model perform the strategies below to avoid overfitting problems. You can reuse the network weights from previous training, often referred to as ``fine tuning``. 
*   **2.3.1** Data augmentation
*   **2.3.2** Dropout
*   **2.3.3** Hyperparameter tuning (e.g. changing learning rate)

> Plot loss and accuracy graphs per epoch side by side for each implemented strategy.

#### 2.3.1 Data augmentation (4 marks)

> Implement at least five different data augmentation techniques that should include both photometric and geometric augmentations. 

> Provide graphs and comment on what you observe.


In [ ]:
# Your code here!

#### 2.3.2 Dropout (4 marks)

> Implement dropout in your model.

> Provide graphs and comment on your choice of proportion used.

In [ ]:
# Your code here!

#### 2.3.3 Hyperparameter tuning (4 marks)

> Use learning rates [0.1, 0.001, 0.0001].

> Provide graphs each for loss and accuracy at three different learning rates in a single graph.

In [ ]:
# Your code here!

In [ ]:
# Your graph

### 3 Model testing [10 marks]
Online evaluation of your model performance on the test set. 

> Prepare the dataloader for the testset.

> Write evaluation code for writing predictions.

> Upload it to Kaggle submission page (6 marks).


#### 3.1 Test class and predictions (4 marks)

> Build a test class, prepare a test dataloader and generate predictions.

Create a PyTorch ```Dataset``` for the unlabeled test data in the test_set folder of the Kaggle competition and generate predictions using your final model. Test data can be downloaded [here](https://www.kaggle.com/competitions/comp5623m-artificial-intelligence/data?select=test_set).

In [ ]:
# Your code here!

#### 3.2 Prepare your submission and upload to Kaggle  (6 marks)

Save all test predictions to a CSV file and submit it to the private class Kaggle competition. **Please save your test CSV file submissions using your student username (the one with letters, e.g., ``sc15jb``, not the ID with only numbers)**, for example, `sc15jb.csv`. That will help us to identify your submissions.

The CSV file must contain only two columns: ‘Id’ and ‘Category’ (predicted class ID) as shown below:

```txt
Id,Category
28d0f5e9_373c.JPEG,2
bbe4895f_40bf.JPEG,18
```

The ‘Id’ column should include the name of the image. It is important to keep the same name as the one on the test set. Do not include any path, just the name of file (with extension). Your csv file must contain 1501 rows, one for each image in test set and 1 row for the headers. [To submit please click here.](https://www.kaggle.com/t/917fe52f6a3c4855880a24b34f26db07)

> You may submit multiple times. We will use your personal top entry for allocating marks for this [6 marks]. 


In [ ]:
# Your code here! 


### 4 Model Fine-tuning/transfer learning on CIFAR10 dataset  [16 marks]

Fine-tuning is a way of applying or utilizing transfer learning. It is a process that takes a model that has already been trained for one given task and then tunes or tweaks the model to make it perform a second similar task. You can perform fine-tuning in the following fashion:

- Train an entire model: Start training model from scratch (large dataset, more computation)

- Train some layers, freeze others: Lower layer features are general (problem independent) while higher layer features are specific (problem dependent – freeze)

- Freeze convolution base and train only last FC layers (small dataset and lower computation) 

> **Configuring your dataset**
   - Download your dataset using ``torchvision.datasets.CIFAR10`` [explained here](https://pytorch.org/vision/stable/generated/torchvision.datasets.CIFAR10.html)
   - Split training dataset into training and validation set similar to above. *Note that the number of categories here is only 10*


In [ ]:
# Your code here! 

> Load pretrained AlexNet from PyTorch - use model copies to apply transfer learning in different configurations

In [ ]:
# Your code here! 

#### 4.1 Apply transfer learning with pretrained model weights (6 marks)


> Configuration 1: No frozen layers

In [ ]:
# Your model changes here - also print trainable parameters

#### 4.2 Fine-tuning model with frozen layers (6 marks)

> Configuration 2: Frozen base convolution blocks

In [ ]:
# Your changes here - also print trainable parameters

#### 4.3 Compare above configurations and comment on performances. (4 marks)

In [ ]:
# Your graphs here and please provide comment in markdown in another cell

### Part II: Multimodal data training [40 marks]


### Motivation 

Through this part of assessment you will:

> 1. Use a Recurrent Neural Network (RNN) to predict the caption of an image from established word vocabularies
> 2. Understand and visualise text predictions for a given image.
> 3. Learn to use transformers for image embedding and transformer-based large language models for text embedding learning. (new)

#### Dataset
This assessment will use a subset of the [COCO "Common Objects in Context" dataset](https://cocodataset.org/) for image caption generation. COCO contains 330K images, of 80 object categories, and at least five textual reference captions per image. Our subset consists of nearly 5070 of these images, each of which has five or more different descriptions of the salient entities and activities, and we will refer to it as COCO_5070.

To download the data:

> 1. **Images and annotations**: download the zipped file provided in the link here as [``COMP5625M_data_assessment_2.zip``](https://leeds365-my.sharepoint.com/:u:/g/personal/scssali_leeds_ac_uk/EWWzE-_AIrlOkvOKxH4rjIgBF_eUx8KDJMPKM2eHwCE0dg?e=DdX62H). 

``Info only:`` To understand more about the COCO dataset you can look at the [download page](https://cocodataset.org/#download). We have already provided you with the "2017 Train/Val annotations (241MB)" but our image subset consists of fewer images compared to orginial COCO dataset. So, no need to download anything from here! 

> 2. **Image meta data**: as our set is a subset of full COCO dataset, we have created a CSV file containing relevant meta data for our particular subset of images. You can download it also from Drive, "coco_subset_meta.csv" at the same link as 1.

#### Submission

You can either submit the same file or make two separate .ipython notebook files zipped in the submission (please name as ``yourstudentusername_partI.ipynb`` and ``yourstudentusername_partII.ipynb``). 

**Final note:**

> **Please include in this notebook everything that you would like to be marked, including figures. Under each section, put the relevant code containing your solution. You may re-use functions you defined previously, but any new code must be in the relevant section.** Feel free to add as many code cells as you need under each section.


The basic principle of our image-to-text model is as pictured in the diagram below, where an Encoder network encodes the input image as a feature vector by providing the outputs of the last fully-connected layer of a pre-trained CNN (we use [ResNet50](https://arxiv.org/abs/1512.03385)). This pretrained network has been trained on the complete ImageNet dataset and is thus able to recognise common objects. 

You can alternatively use the COCO trained pretrained weights from [PyTorch](https://pytorch.org/vision/stable/models.html). One way to do this is use the "FasterRCNN_ResNet50_FPN_V2_Weights.COCO_V1" but use e.g., "resnet_model = model.backbone.body". Alternatively, you can use the checkpoint from your previous coursework where you fine-tuned to COCO dataset. 

These features are then fed into a Decoder network along with the reference captions. As the image feature dimensions are large and sparse, the Decoder network includes a linear layer which downsizes them, followed by *a batch normalisation layer to speed up training*. Those resulting features, as well as the reference text captions, are then passed into a recurrent network (we will use an **RNNs** in this assessment). 


### Sample RNN image captioning
The reference captions used to compute loss are represented as numerical vectors via an **embedding layer** whose weights are learned during training.

<!-- ![Encoder Decoder](comp5625M_figure.jpg) -->

<div>
<center><img src="comp5625M_figure_v3.jpg" width="1000"/></center>
</div>


### Instructions for creating vocabulary 

A helper function file ``helperDL.py`` is provided that includes all the functions that will do the following for you. You can easily import these functions in the exercise, most are already done for you!  

> 1. Extracting image features (a pre-trained checkpoint is provided ``coco_features_{cpu/gpu}.pt`` for you to download and use it for training your RNN)
> 2. Text preparation of training and validation data is provided

In [106]:
import torch
import torch.nn as nn
from torchvision import transforms
import torchvision.models as models
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import matplotlib.pyplot as plt
from collections import Counter
import os
import numpy as np

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:
# Load the feature map provided to you
features_map = torch.load('coco_features_'+(device.type)+'.pt', map_location=device)

#### 5 Train an RNN-based image captioning [20 marks]

Most of the background codes are provided to you. You only will require to write down the code based on your understanding to:

> 5.1 Design a RNN-based decoder. (8 marks)

> 5.2 Train your model with precomputed features. (6 Marks)

> 5.3 Train using new parameters. (6 Marks)




#### Text data preparation

In [ ]:
import pandas as pd
# captions.csv is provided to you
# coco/images/ folder is provided to you

captions_path = './'
image_path  = "coco/images/"
df = pd.read_csv(captions_path+"captions.csv")

#visualise
df.head(10)

In [ ]:
# Note: For RNN you will need to clean the text, for e.g., converting all uppercases to lowercases 
# gen_clean_captions_df fuction will do this for you.

df["clean_caption"] = ""
from helperDL import gen_clean_captions_df
df_clean = gen_clean_captions_df(df)
df_clean.head(10)

In [ ]:
##### Spilt your training, validation and test dataset with indexes to each set
from helperDL import split_ids
train_id, valid_id, test_id = split_ids(df['id'].unique())
print("training:{}, validation:{}, test:{}".format(len(train_id), len(valid_id), len(test_id)))

In [112]:
train_set = df.loc[df['id'].isin(train_id)]
valid_set = df.loc[df['id'].isin(valid_id)]
test_set = df.loc[df['id'].isin(test_id)]

In [113]:
#### word 2 an integer ID - build vocabulary
class Vocabulary(object):
    """ Simple vocabulary wrapper which maps every unique word to an integer ID. """
    def __init__(self):
        # intially, set both the IDs and words to dictionaries with special tokens
        self.word2idx = {'<pad>': 0, '<unk>': 1, '<end>': 2}
        self.idx2word = {0: '<pad>', 1: '<unk>', 2: '<end>'}
        self.idx = 3

    def add_word(self, word):
        # if the word does not already exist in the dictionary, add it
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            # increment the ID for the next word
            self.idx += 1

    def __call__(self, word):
        # if we try to access a word not in the dictionary, return the id for <unk>
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

In [ ]:
### build vocabulariy for each set - train, val and test 
# you will be using to create dataloaders
from helperDL import build_vocab

# create a vocab instance
vocab = Vocabulary()
vocab_train = build_vocab(train_id, df_clean, vocab)
vocab_valid = build_vocab(valid_id, df_clean, vocab)
vocab_test = build_vocab(test_id, df_clean, vocab)

vocab = vocab_train # using only training samples as vocabulary as instructed above
print("Total vocabulary size: {}".format(len(vocab_train)))


In [ ]:
# They can also join the train and valid captions but they will need to run vocabulary after concatenation
import numpy as np
vocab = build_vocab(np.concatenate((train_id, valid_id), axis=0), df_clean, vocab) #---> overrighting
len(vocab)

Instantiate a ```DataLoader``` for your image feature and caption dataset. ``helperDL.py`` file includes all the required functions

We need to overwrite the default PyTorch collate_fn() because our 
ground truth captions are sequential data of varying lengths. The default
collate_fn() does not support merging the captions with padding.

You can read more about it here:
https://pytorch.org/docs/stable/data.html#dataloader-collate-fn. 


In [ ]:
from helperDL import EncoderCNN  
model = EncoderCNN() 
print(model)

In [ ]:
''' Load the saved image feature maps and trained model ("provided to you") '''

# ---> your entry here (make sure that the path is correct - load the coco_features_{cpu/gpu}.pt)
features = torch.load("coco_features.pt")

# ---> load the model ckpt and udate the model state dict of the base model
# your entry here (make sure that the path is correct)
checkpoint = torch.load("ckpt file here") 
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
##### Preparing the train, val and test dataloaders
from helperDL import COCO_Features
from helperDL import caption_collate_fn

# Create a dataloader for train
dataset_train = COCO_Features(
    df=train_set,
    vocab=vocab,
    features=features_map,
)

train_loader = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=32,
    shuffle=True,
    num_workers=0, # may need to set to 0
    collate_fn=caption_collate_fn, # explicitly overwrite the collate_fn
)

# Create a dataloader for valid
dataset_valid = COCO_Features(
    df=valid_set,
    vocab=vocab,
    features=features_map,
)

valid_loader = torch.utils.data.DataLoader(
    dataset_valid,
    batch_size=32,
    shuffle=True,
    num_workers=0, # may need to set to 0
    collate_fn=caption_collate_fn, # explicitly overwrite the collate_fn
)

In [120]:
# Configuration file - You will be asked below to train your RNN using various configurations and compare.
# --> Please change these numbers as required. 
# --> Please comment on changes that you do.

class cfg:
    EMBED_SIZE = 256
    HIDDEN_SIZE = 512
    NUM_LAYERS = 1
    LR = 0.001
    NUM_EPOCHS = 5
    LOG_STEP = 10
    MAX_SEQ_LEN = 37

#### 5.1 Design a RNN-based decoder (8 marks)

Read through the ```DecoderRNN``` model below. First, complete the decoder by adding an ```RNN``` layer to the decoder where indicated, using [the PyTorch API as reference](https://pytorch.org/docs/stable/nn.html#rnn).

Keep all the default parameters except for ```batch_first```, which you may set to True.

In particular, understand the meaning of ```pack_padded_sequence()``` as used in ```forward()```. Refer to the [PyTorch ```pack_padded_sequence()``` documentation](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pack_padded_sequence.html).

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, vocab_size, embed_size=cfg.EMBED_SIZE, hidden_size=cfg.HIDDEN_SIZE, num_layers=cfg.NUM_LAYERS, max_seq_length=cfg.MAX_SEQ_LEN):
        """Set the hyper-parameters and build the layers."""
        super(DecoderRNN, self).__init__()
        # we want a specific output size, which is the size of our embedding, so
        # we feed our extracted features from the last fc layer (dimensions 1 x 2048)
        # into a Linear layer to resize
        self.resize = nn.Linear(2048, embed_size)
        
        # batch normalisation helps to speed up training
        self.bn = nn.BatchNorm1d(embed_size, momentum=0.01)

        # Embedding layer
        self.embed = nn.Embedding(vocab_size, embed_size)

        # ---> RNN (use nn.RNN function passing arguments that you computed above, use batch_first=True as an argument)
        self.rnn =                                       #... your code here

        self.linear = nn.Linear(hidden_size, vocab_size)
        self.max_seq_length = max_seq_length
        

    def forward(self, features, captions, lengths):
        """Decode image feature vectors and generates captions."""

        # a) compute the embedding of caption (Complete the code!)
        embeddings =
        # b) resize the image features (Complete the code!)
        im_features = 
        # c) apply batch norm on image the resized feature (Complete the code!)
        im_features =
        # d) concatenate the image feature and the embeddings - use unsqueeze(1) to make the appropriate  size
        embeddings = torch.cat((im_features.unsqueeze(1), embeddings), 1)
        # e) use pack_padded_sequence 
        packed = pack_padded_sequence(embeddings, lengths, batch_first=True) 
        hiddens, _ = self.rnn(packed)
        outputs = self.linear(hiddens[0])
        return outputs
    

    def sample(self, features, states=None):
        """Generate captions for given image features using greedy search."""
        sampled_ids = []

        inputs = self.bn(self.resize(features)).unsqueeze(1)
        for i in range(self.max_seq_length):
            hiddens, states = self.rnn(inputs, states)  # hiddens: (batch_size, 1, hidden_size)
            outputs = self.linear(hiddens.squeeze(1))   # outputs:  (batch_size, vocab_size)
            _, predicted = outputs.max(1)               # predicted: (batch_size)
            sampled_ids.append(predicted)
            inputs = self.embed(predicted)              # inputs: (batch_size, embed_size)
            inputs = inputs.unsqueeze(1)                # inputs: (batch_size, 1, embed_size)
        sampled_ids = torch.stack(sampled_ids, 1)       # sampled_ids: (batch_size, max_seq_length)
        return sampled_ids

In [ ]:
# instantiate decoder
decoder = DecoderRNN(len(vocab), embed_size=cfg.EMBED_SIZE, hidden_size=cfg.HIDDEN_SIZE, num_layers=cfg.NUM_LAYERS).to(device)

##### 5.2 Train your model with precomputed features [6 marks]

Train the decoder by passing the features, reference captions, and targets to the decoder, then computing loss based on the outputs and the targets. Note that when passing the targets and model outputs to the loss function, the targets will also need to be formatted using ```pack_padded_sequence()```.

We recommend a batch size of around 64 (though feel free to adjust as necessary for your hardware).

**We strongly recommend saving a checkpoint of your trained model after training so you don't need to re-train multiple times.**

Display a graph of training and validation loss over epochs to justify your stopping point.

In [ ]:
# Define: loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(decoder.parameters(), lr=cfg.LR)

In [ ]:
def train():
    # train the models
    total_step = len(train_loader)
    total_step_v = len(valid_loader)
    stats = np.zeros((cfg.NUM_EPOCHS,2))
    print(stats.shape)
    total_loss = 0
    for epoch in range(cfg.NUM_EPOCHS):
        for i, (features_, captions_, lengths_) in enumerate(train_loader):

            # set mini-batch dataset
            features_ = features_.to(device)
            captions_ = captions_.to(device)
            targets = pack_padded_sequence(captions_, lengths_, batch_first=True)[0]
            decoder.zero_grad()

            # forward, backward and optimize
            outputs = decoder(features_, captions_, lengths_)
            loss = criterion(outputs, targets)

            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()

            # print stats
            if i % cfg.LOG_STEP == 0:
                print(f"Epoch [{epoch+1}/{cfg.NUM_EPOCHS}], Step [{i}/{total_step}], Loss: {loss.item():.4f}")
        
        stats[epoch,0] = round(total_loss/total_step,3)
        total_loss = 0
        decoder.eval()
        with torch.no_grad():  
            for i, (features_, captions_, lengths_) in enumerate(valid_loader):

                # set mini-batch dataset
                features_ = features_.to(device)
                captions_ = captions_.to(device)
                targets = pack_padded_sequence(captions_, lengths_, batch_first=True)[0]

                # forward, backward and optimize
                outputs = decoder(features_, captions_, lengths_)

                loss = criterion(outputs, targets)

                total_loss += loss.item()
        
        stats[epoch,1] = round(total_loss/total_step_v,3)
        total_loss = 0
        # print stats
        print("="*30)
        print(f"Epoch [{epoch+1}/{cfg.NUM_EPOCHS}], Train_Loss: {stats[epoch,0]}, Valid_Loss: {stats[epoch,1]}")
        print("="*30)
        decoder.train()

    return stats

In [ ]:
def plot_stats(stats, figure_name="coco_train_vocab_only.png"):
    fig = plt.figure()
    plt.plot(stats[:,0], 'r', label = 'training', )
    plt.plot(stats[:,1], 'g', label = 'validation' )
    plt.legend()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title(f"COCO dataset- train vocab only #vocab={len(vocab)}")
    fig.savefig(figure_name)
    plt.show()

#### Train using default parameters

In [ ]:
stats = train()
plot_stats(stats, "coco_default_parameters.png")

#### 5.3 Train using new parameters (6 Marks)

> In CFG class change --> NUM_LAYERS = 2 and  EMBED_SIZE = 512


In [ ]:
#your code here!

In [ ]:
# save the best model after training
#your code here! e.g.: decoder_ckpt = torch.save(decoder, "coco_subset_assessment_decoder.ckpt")



### 6 Test prediction and evaluation [8 marks] 

#### 6.1 Generate predictions on test data (4 marks)

Display 4 sample test images containing different objects, along with your model’s generated captions and all the reference captions for each.

> Remember that everything **displayed** in the submitted notebook and .html file will be marked, so be sure to run all relevant cells.

In [ ]:
from helperDL import COCOImagesDataset

data_transform = transforms.Compose([ 
    transforms.Resize(224),     
    transforms.CenterCrop(224), 
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),   # using ImageNet norms
                         (0.229, 0.224, 0.225))])

dataset_test = COCOImagesDataset(
    df=test_set,
    transform=data_transform,
)
test_loader = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=32,
    shuffle=False,
    num_workers=0)

In [ ]:
model.eval()
decoder.eval() # generate caption, eval mode to not influence batchnormncoder.eval()

In [ ]:
# getting functions from helperDL.py
from helperDL import timshow
from helperDL import decode_caption

In [ ]:
IMAGES_TO_SHOW = 4
idx = 0
with torch.no_grad():
    for i, (image,filename) in enumerate(test_loader):
        
        # your code here --->







        
        print(f"GENERATED: \n")
        print("REFERENCES:")

        print("===================================\n")


        timshow(image[0].cpu())
        idx +=1
        if idx == IMAGES_TO_SHOW:
            break

#### 6.2 Caption evaluation using cosine similarity (4 marks)

The cosine similarity measures the cosine of the angle between two vectors in n-dimensional space. The smaller the angle, the greater the similarity.

To use the cosine similarity to measure the similarity between the generated caption and the reference captions: 

* Find the embedding vector of each word in the caption 
* Compute the average vector for each caption 
* Compute the cosine similarity score between the average vector of the generated caption and average vector of each reference caption
* Compute the average of these scores 

Calculate the cosine similarity using the model's predictions over the whole test set. 

In [ ]:
# your code here

## 7 Use language models for image captioning [12 marks]

Description: A large language model (LLM) is a type of artificial intelligence (AI) model that uses deep learning to perform natural language processing (NLP) tasks.

One way to train on multimodal (image and text data) is CLIP (Contrastive Language-Image Pre-Training). CLIP is a neural network trained on a variety of (image, text) pairs and can take CNNs or transformers for encoding images and widely accepted LLMs for text embedding. Paper here: https://arxiv.org/pdf/2103.00020

CLIP is trained on flickr 30K dataset. First we will try on flickr 8k data then on COCO.

> Most codes are provided for you. 
> Small model and large pre-trained models are given to you and also we have provided the configS and configL that are required to run these models.

> **Your task is to:**

    - a. Understand how the smaller model perform on the flickr 8k and COCO dataset (2 images each) [4 marks]

    - b. Understand how the larger model perform on the flickr 8k and COCO dataset (2 images each) [4 marks]
    
    - c. Finally, understand how the CLIP model trained on a larger curated COCO dataset generalises to the COCO dataset (use same 2 images from above) [4 marks]

You can find the link to download small and large pretrained CLIP models also here: [CLIP_S_L_Models](https://leeds365-my.sharepoint.com/:u:/g/personal/scssali_leeds_ac_uk/ERX5QK77b81Jv4K_5ff58ekBsXIs6EAL7tC4Ge1COMMRrA?e=gnzmX7)

In [ ]:
import os

#configuration for the smaller model (slightly over 1 GB model weight file)

class configS:
    """
    Project's main config.
    """

    clip_model: str = "openai/clip-vit-base-patch32"
    text_model: str = "gpt2"
    seed: int = 100
    num_workers: int = 2
    train_size: int = 0.84
    val_size: int = 0.13
    epochs: int = 150
    lr: int = 3e-3
    k: float = 0.33
    batch_size_exp: int = 6
    ep_len: int = 4
    num_layers: int = 6
    n_heads: int = 16
    forward_expansion: int = 4
    max_len: int = 40
    dropout: float = 0.1
    weights_dir: str = os.path.join("weights", "small")

In [ ]:
#configuration for the larger model (nearly 3 GB model weight file)
class configL:
    """
    Project's main config - large.
    """

    clip_model: str = "openai/clip-vit-large-patch14"
    text_model: str = "gpt2-medium"
    seed: int = 100
    num_workers: int = 2
    train_size: int = 0.84
    val_size: int = 0.13
    epochs: int = 120
    lr: int = 5e-3
    k: float = 0.3
    batch_size_exp: int = 5
    ep_len: int = 4
    num_layers: int = 5
    n_heads: int = 16
    forward_expansion: int = 4
    max_len: int = 40
    dropout: float = 0.08
    weights_dir: str = os.path.join("weights", "large")

In [ ]:
from model_CLIP import Net # function is given to you

model = Net(
    clip_model=configS.clip_model,
    text_model=configS.text_model,
    ep_len=configS.ep_len,
    num_layers=configS.num_layers,
    n_heads=configS.n_heads,
    forward_expansion=configS.forward_expansion,
    dropout=configS.dropout,
    max_len=configS.max_len,
    device=device,
)

#use small model
checkpoint = torch.load('model_S.pt', map_location=device) 
model.load_state_dict(checkpoint, strict=False)
# model.load_state_dict(checkpoint, strict=False)

> Try on COCO dataset with the small model only - Comment on your observation.

In [ ]:
model.eval()
from PIL import Image


# Output on two images of your choice from 'flickr 8k' 
# img = Image.open('Images/55473406_1d2271c1f2.jpg')
# img = Image.open('Images/260828892_7925d27865.jpg')
# Your code here:


# Output on two images of your choice from 'COCO' dataset
# img = Image.open(image_path + './000000000030.jpg')
# Your code here:



temperature = 1.0
with torch.no_grad():
    caption, _ = model(img, temperature)

plt.imshow(img)
plt.title(caption)
plt.axis("off")

<--- Your comment here --->

> Repeat the above with the larger model "model_L" - use large model configuration. Comment on your observations.

In [ ]:
###Your code here!



<--- Your comment here --->

> Try on COCO dataset with the large model only - Comment on your observation. What would you have to do to improve network performance?

In [ ]:
###Your code here!

<- Your comment here ->


##### Now try on the CLIP fine-tuned on High-Level dataset. 

The High-Level (HL) dataset aligns object-centric descriptions from COCO with high-level descriptions crowdsourced along 3 axes: scene, action, rationale.

The HL dataset contains 14997 images from COCO and a total of 134973 crowdsourced captions (3 captions for each axis) aligned with ~749984 object-centric captions from COCO.

https://huggingface.co/datasets/michelecafagna26/hl

In [ ]:
#run these to install CLIPClap - fine tuned on COCO

!pip install git+https://github.com/michelecafagna26/CLIPCap.git

!git lfs install 
!git clone https://huggingface.co/michelecafagna26/clipcap-base-captioning-ft-hl-scenes

In [ ]:
from clipcap import ClipCaptionModel
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import clip
import requests
from PIL import Image

model_path = "clipcap-base-captioning-ft-hl-scenes/pytorch_model.pt" # change accordingly

# load clip
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
prefix_length = 10

# load ClipCap
model = ClipCaptionModel(prefix_length, tokenizer=tokenizer)
model.from_pretrained(model_path)
model = model.eval()
model = model.to(device)


In [ ]:

# load the image
# raw_image = Image.open('Images/260828892_7925d27865.jpg').convert('RGB')
# raw_image = Image.open(image_path + './000000000030.jpg').convert('RGB')

# Your code here --->

# extract the prefix
img = preprocess(raw_image).unsqueeze(0).to(device)
with torch.no_grad():
    prefix = clip_model.encode_image(img).to(
        device, dtype=torch.float32
    )
    prefix_embed = model.clip_project(prefix).reshape(1, prefix_length, -1)

# generate the caption   
caption = model.generate_beam(embed=prefix_embed)[0]

plt.imshow(raw_image)
plt.title(caption)
plt.axis("off")

<--- Your comment here --->

<Your answer here>

##### Thank you for completing the practical assessment - if you have any question, please ask on teams channel or attend lab sessions on Wednesdays and Thursdays.